In [1]:
from skimage.transform import resize
import random
import numpy as np
import matplotlib.pyplot as plt
import math
import random

# Catch Environment (from the assignment code)
from Catch import CatchEnv

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torch.nn import Module
from torch.nn import Conv2d
from torch.nn import Linear
from torch.nn import MaxPool2d
from torch.nn import ReLU
from torch.nn import LogSoftmax
from torch import flatten



c:\Users\danie\miniconda3\envs\footballtracking\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Memory buffer, to store the trajectories

In [2]:

from collections import namedtuple, deque

Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))

class MemoryBuffer(object):
    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity) # iterable deque data structure

    def push(self, *args):
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

### Make a Deep Q-Network

In [33]:

# the network should have 3 outputs, one for each of the possible moves

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# I CHANGED A BUNCH OF STUFF
"""
This is broken as fuck, do not trust it, I copied it from somewhere :/
probably best to start over on this DQN model
"""

class DQN(nn.Module):

    def __init__(self, number_of_actions):
        super(DQN, self).__init__()
        # 1 input image channel (black & white), 6 output channels, 5x5 square convolution
        # conv layers
        self.conv1 = torch.nn.Conv2d(4, 32, 8, stride=4)   # modify input shape to match your input data size
        self.conv2 = torch.nn.Conv2d(32, 64, 4, stride=2)
        self.conv3 = torch.nn.Conv2d(64, 64, 3)
        # fully connected layers
        self.fc1 = torch.nn.Linear(7 * 7 * 64, 512)
        self.fc2 = torch.nn.Linear(512, number_of_actions)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, (2, 2))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [34]:
class Agent(object):
    def __init__(self, env, eps_start, eps_min, eps_decay, memory, batch_size, optimizer, gamma):
        self.gamma = gamma
        self.optimizer = optimizer
        self.batch_size = batch_size
        self.memory = memory
        self.env = env
        self.eps = eps_start
        self.eps = eps_min
        self.eps = eps_decay

        self.model = DQN(env.get_num_actions()).to(device)
        self.optimizer = optim.Adam(self.model.parameters(), lr=1e-4)

    def select_action(self, state):              #NOT REALLY SURE IF THIS IS DOING WHAT IT NEEDS TO
        if np.random.rand() <= self.eps:
            return random.randrange(self.env.get_num_actions())

        state = np.expand_dims(state, axis=0)
        
        print(state.shape)
        state = torch.from_numpy(state).float().to(device)
        q_values = self.model(state)
        action = torch.argmax(q_values, dim=1).item()
        return action

    def replay(self):
        if len(self.memory) < self.batch_size:
            return

        batch = random.sample(self.memory, self.batch_size)
        states, actions, rewards, next_states, dones = zip(*batch)

        states = torch.from_numpy(np.array(states)).float().to(device)
        actions = torch.from_numpy(np.array(actions)).unsqueeze(1).to(device) # maybe take out the unsqueeze
        rewards = torch.from_numpy(np.array(rewards)).float().to(device)
        next_states = torch.from_numpy(np.array(next_states)).float().to(device)
        terminals = torch.from_numpy(np.array(terminals)).float().to(device)

        q_values = self.model(states)
        q_values = q_values.gather(1, actions.unsqueeze(1)).squeeze(1)
        next_q_values = self.model(next_states).max(1)[0]
        expected_q_values = rewards + (1 - terminals) * self.gamma * next_q_values

        loss = F.mse_loss(q_values, expected_q_values.detach())
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        if self.eps > self.eps_min:
            self.eps *= self.eps_decay

    def load(self, name):
        self.model.load_state_dict(torch.load(name))

    def save(self, name):
        torch.save(self.model.state_dict(), name)

## Loops

In [31]:
# BATCH_SIZE is the number of transitions sampled from the replay buffer
# GAMMA is the discount factor as mentioned in the previous section
# EPS_START is the starting value of epsilon
# EPS_END is the final value of epsilon
# EPS_DECAY controls the rate of exponential decay of epsilon, higher means a slower decay
# TAU is the update rate of the target network
# LR is the learning rate of the ``AdamW`` optimizer
BATCH_SIZE = 128
GAMMA = 0.99
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 1000
TAU = 0.005
LR = 1e-4


# initialising the target and policy networks:
env = CatchEnv()

# Get number of actions from gym action space
n_actions = env.get_num_actions()

# Get the number of state observations
env.reset()
state, reward, terminal = env.step(random.randint(0, 2))
n_observations = len(state)

policy_network = DQN(n_actions).to(device)
target_network = DQN(n_actions).to(device)

target_network.load_state_dict(policy_network.state_dict())

criterion = nn.MSELoss() # this one is inside the agent I think
optimizer = optim.AdamW(policy_network.parameters(), lr=LR, amsgrad=True)

memoryBuffer = MemoryBuffer(100)


agent = Agent(env, EPS_START, EPS_END, EPS_DECAY, memoryBuffer, BATCH_SIZE, optimizer, GAMMA)


'\n\n# BATCH_SIZE is the number of transitions sampled from the replay buffer\n# GAMMA is the discount factor as mentioned in the previous section\n# EPS_START is the starting value of epsilon\n# EPS_END is the final value of epsilon\n# EPS_DECAY controls the rate of exponential decay of epsilon, higher means a slower decay\n# TAU is the update rate of the target network\n# LR is the learning rate of the ``AdamW`` optimizer\nBATCH_SIZE = 128\nGAMMA = 0.99\nEPS_START = 0.9\nEPS_END = 0.05\nEPS_DECAY = 1000\nTAU = 0.005\nLR = 1e-4\n\n\n# initialising the target and policy networks:\nenv = CatchEnv()\n\n# Get number of actions from gym action space\nn_actions = env.get_num_actions()\n\n# Get the number of state observations\nenv.reset()\nstate, reward, terminal = env.step(random.randint(0, 2))\nn_observations = len(state)\n\npolicy_network = DQN(n_actions).to(device)\ntarget_network = DQN(n_actions).to(device)\n\ntarget_network.load_state_dict(policy_network.state_dict())\n\ncriterion = n

In [35]:

if torch.cuda.is_available():
    num_episodes = 500
else:
    num_episodes = 10

env = CatchEnv()

for episode in range(num_episodes):
    state = env.reset()
    state = np.squeeze(state)
    #state, reward, terminal = env.step(random.randint(0, 2))
    terminal = False
    total_reward = 0
    while not terminal:
        # agent interacts with the environment
        action = agent.select_action(state)    
        next_state, reward, terminal = env.step(action)
        next_state = np.squeeze(next_state)
        
        if terminal:
            next_state = None

        # add trajectory to memory buffer
        agent.memory.push(state, action, next_state, reward)

        # Update the total reward
        total_reward += reward
        # move onto the next state
        state = next_state

        # Sample a batch of transitions from the replay buffer
        agent.replay()
            
        # Update the target network
        if episode % 10 == 0: ## MAYBE USE THE AGENTS FUNCTION
            target_network.load_state_dict(policy_network.state_dict())



    print ("End of the episode")
    print ("  Reward obtained by the agent : {}". format(total_reward))

(84, 84, 4)
doing conv1
torch.Size([1, 84, 84, 4])


RuntimeError: permute(sparse_coo): number of dimensions in the tensor input does not match the length of the desired ordering of dimensions i.e. input.dim() = 5 is not equal to len(dims) = 3

arquitectures for the grid search